In [2]:
import os
from data_process import get_MNIST_dataloader
from MNIST_network import MNIST
import paddle.nn.functional as F
import paddle
train_loader, test_loader = get_MNIST_dataloader()

e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\fluid\reader.py:486: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  warnings.warn(


In [3]:
class Trainer(object):
    def __init__(self, model_path, model, optimizer):
        self.model_path = model_path   # 模型存放路径 
        self.model = model             # 定义的模型
        self.optimizer = optimizer     # 优化器

    def save(self):
        # 保存模型
        paddle.save(self.model.state_dict(), self.model_path)

    def train_step(self, data):
        images, labels = data
        # 前向计算的过程
        predicts = self.model(images)
        # 计算损失
        loss = F.cross_entropy(predicts, labels)
        avg_loss = paddle.mean(loss)
        # 后向传播，更新参数的过程
        avg_loss.backward()
        self.optimizer.step()
        self.optimizer.clear_grad()
        return avg_loss

    def train_epoch(self, datasets, epoch):
        self.model.train()
        for batch_id, data in enumerate(datasets()):
            loss = self.train_step(data)
            # 每训练了1000批次的数据，打印下当前Loss的情况
            if batch_id % 500 == 0:
                print("epoch_id: {}, batch_id: {}, loss is: {}".format(epoch, batch_id, loss.numpy()))

    def train(self, train_datasets, start_epoch, end_epoch, save_path):
        if not os.path.exists(save_path):
            os.makedirs(save_path)
        for i in range(start_epoch, end_epoch):
            self.train_epoch(train_datasets, i)

            paddle.save(self.optimizer.state_dict(), './{}/mnist_epoch{}'.format(save_path,i)+'.pdopt')
            paddle.save(self.model.state_dict(), './{}/mnist_epoch{}'.format(save_path,i)+'.pdparams')

        self.save()

In [4]:
#在使用GPU机器时，可以将use_gpu变量设置成True
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

paddle.seed(1024)

epochs = 3
BATCH_SIZE = 32
model_path = './mnist.pdparams'

model = MNIST()

total_steps = (int(50000//BATCH_SIZE) + 1) * epochs
lr = paddle.optimizer.lr.PolynomialDecay(learning_rate=0.01, decay_steps=total_steps, end_lr=0.001)
optimizer = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters())

trainer = Trainer(
    model_path=model_path,
    model=model,
    optimizer=optimizer
)

trainer.train(train_datasets=train_loader, start_epoch = 0, end_epoch = epochs, save_path='checkpoint')

epoch_id: 0, batch_id: 0, loss is: [4.078209]
epoch_id: 0, batch_id: 500, loss is: [0.33502007]
epoch_id: 1, batch_id: 0, loss is: [0.20112367]
epoch_id: 1, batch_id: 500, loss is: [0.10331787]
epoch_id: 2, batch_id: 0, loss is: [0.07159668]
epoch_id: 2, batch_id: 500, loss is: [0.2904286]


In [5]:
paddle.seed(1024)


epochs = 3
BATCH_SIZE = 32
model_path = './mnist_retrain.pdparams'

model = MNIST()
total_steps = (int(50000/BATCH_SIZE) +1 )* epochs
lr = paddle.optimizer.lr.PolynomiaDecay(learning_rate = 0.01, decay_steps=total_steps,end_lr= 0.001)
optimizer = paddle.optimizer.Adam(learning_rate=lr, parameters=model.parameters())

params_dict = paddle.load('./checkpoint/mnist_epoch0.pdparams')
opt_dict = paddle.load('./checkpoint/mnist_epoch0.pdopt')

# 加载参数到模型
model.set_state_dict(params_dict)
optimizer.set_state_dict(opt_dict)

trainer = Trainer(
    model_path=model_path,
    model=model,
    optimizer=optimizer
)
# 前面训练模型都保存了，这里save_path设置为新路径，实际训练中保存在同一目录就可以
trainer.train(train_datasets=train_loader,start_epoch = 1, end_epoch = epochs, save_path='checkpoint_con')

AttributeError: module 'paddle.optimizer.lr' has no attribute 'PolynomiaDecay'

In [ ]:
# 这里我做一些修改